In [4]:
import os
import csv
import cv2
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

from random import shuffle

from sklearn.model_selection import train_test_split

from keras.layers.advanced_activations import ELU

In [5]:
from keras.models import Sequential, Model
from keras.layers.core import Dense, Flatten, Lambda, Dropout, Activation
from keras.layers.core import Lambda
from keras.layers import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Cropping2D

In [10]:
images = []
measurements = []
samples = []
img = []
with open('driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
        
for line in samples:
    image = 'data/IMG/'+line[0].split('/')[-1]
    image = mpimg.imread(image)
    image = augment_brightness_camera_images(image)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    image = augment_brightness_camera_images(image)
    images.append(image)
    measurements.append(measurement)
    
    image = 'data/IMG/'+line[1].split('/')[-1]
    image = mpimg.imread(image)
    image = augment_brightness_camera_images(image)
    images.append(image)
    measurement = float(line[3]) + 0.25
    measurements.append(measurement)
    image = augment_brightness_camera_images(image)
    images.append(image)
    measurements.append(measurement)
    
    image = 'data/IMG/'+line[2].split('/')[-1]
    image = mpimg.imread(image)
    images.append(image)
    measurement = float(line[3]) - 0.25
    measurements.append(measurement)
    image = augment_brightness_camera_images(image)
    images.append(image)
    measurements.append(measurement)
#train_samples, validation_samples = train_test_split(samples, test_size=0.2)

X_train = np.array(images)
y_train = np.array(measurements)

%matplotlib inline

MemoryError: 

In [9]:
def augment_brightness_camera_images(image):
    '''
    :param image: Input image
    :return: output image with randomly adjusted brightness
    '''

    # convert to HSV so that its easy to adjust brightness
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)

    # randomly generate the brightness reduction factor
    # Add a constant so that it prevents the image from being completely dark
    random_bright = .25+np.random.uniform()

    # Apply the brightness reduction to the V channel
    image1[:,:,2] = image1[:,:,2]*random_bright

    # convert to RBG again
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

In [17]:
def my_generator(samples, batch_size=32):
    N = (len(samples)//batch_size)*batch_size  # make the number of samples in 'lines' a multiple of batch_size

    X_batch = np.zeros((batch_size, 160, 320, 3), dtype=np.float32)
    y_batch = np.zeros((batch_size,), dtype=np.float32)
    
    while True:
        for i in range(0, N, batch_size):
            batch_start = i
            batch_stop = i + batch_size
            batch_lines = samples[batch_start:batch_stop]
            images = []
            measurements = []
            for j, line in enumerate(batch_lines):
                #choice = random.choice([0,1,2])
                image = 'data/IMG/'+line[0].split('/')[-1]
                image = mpimg.imread(image)
                #crop the image
                #imageBGR = image[50:135,:,:]
                # apply subtle blur
                '''imageBGR = cv2.GaussianBlur(imageBGR, (3,3), 0)
                
                imageBGR = cv2.resize(imageBGR,(320, 160), interpolation = cv2.INTER_AREA)
                image = cv2.cvtColor(imageBGR,cv2.COLOR_BGR2RGB)'''
                '''if choice == 0:
                    steering = float(line[3])
                
                if choice == 1:
                    steering = float(line[3]) + 0.25
                
                if choice == 2: 
                    steering = float(line[3]) - 0.25'''
                
                image = augment_brightness_camera_images(image)
                
                #images.append(image)
                #measurements.append(steering)
                steering = float(line[3])
                
                X_batch[j] = image
                y_batch[j] = steering
                
            flip_indices = random.sample(range(X_batch.shape[0]), int(X_batch.shape[0] / 2))
            X_batch[flip_indices] = X_batch[flip_indices, :, ::-1, :]
            y_batch[flip_indices] = -y_batch[flip_indices]
                                             
            yield sklearn.utils.shuffle(X_batch, y_batch)


In [7]:
#NVIDIA

# compile and train the model using the generator function
#train_generator = my_generator(train_samples, batch_size=32)
#validation_generator = my_generator(validation_samples, batch_size=32)

ch, row, col = 3, 160, 320

model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(row, col, ch)))
model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=(row,col,ch)))
#model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))
model.add(Flatten())
model.add(Dense(1000))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(500))
model.add(Dense(100))
#model.add(Dropout(0.5))
model.add(Dense(50))
#model.add(Dropout(0.5))
model.add(Dense(20))
#model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Dense(1))
#model.summary()

model.compile(loss = 'mse', optimizer= 'adam')

model.fit(X_train, y_train, validation_split = 0.2, shuffle = True, nb_epoch = 3)

#model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=4)

model.save('model1.h5')

#model.summary()

Train on 19286 samples, validate on 4822 samples
Epoch 1/3
19286/19286 [==============================] - 56s - loss: 0.0258 - val_loss: 0.0250
Epoch 2/3
19286/19286 [==============================] - 48s - loss: 0.0208 - val_loss: 0.0237
Epoch 3/3
19286/19286 [==============================] - 47s - loss: 0.0187 - val_loss: 0.0254


In [10]:
float("9.4321e-08")
"{:.8f}".format(float("9.4321e-08"))

'0.00000009'

In [7]:
model.compile(loss='mse', optimizer='adam')
model.fit(X_train,y_train, validation_split=0.2, shuffle=True,nb_epoch=1)
model.save('model.h5')

NameError: name 'X_train' is not defined